# Notebook 기본 세팅

In [1]:
# Constant 선언

# 프로젝트 루트 디렉토리를 식별하기 위한 마커 파일 이름
ROOT_MARKER = "pyproject.toml"

# 한글 표시를 위한 나눔바른고딕 폰트 파일 이름
# matplotlib 의 font_manager 에 실제 폰트 파일의 위치를 넣어주어야 한다.
KOREAN_FONT_FILE = "NanumBarunGothic.ttf"

# matplotlib 에서는 font-family 의 이름으로 font 를 설정한다.
# 그래서 font 파일 그 자체가 아니라, 그 파일의 family 이름을 적어준다.
KOREAN_FONT_FAMILY = "NanumBarunGothic"

# 참고
# Font Family 와 Font File 의 차이는,
# Font Family 는 비슷한 디자인 특성을 공유하는 글꼴 그룹을 의미한다.
#
# 예를 들어 '나눔바른고딕' 폰트 패밀리는 일반(Regular), 굵게(Bold), 기울임(Italic) 등 여러 스타일을 포함할 수 있다.
# 반면, 폰트 파일(.ttf, .otf 등)은 이러한 폰트의 하나의 스타일이 저장된 실제 파일이다.
#
# 이 프로젝트에서는 폰트 용량을 줄이기 위해 일반(Regular) 인 NanumBarunGothic.ttf 만 사용한다.

In [2]:
# 프로젝트 root 를 sys.path 에 추가해서 import 구문을 사용하기 쉽게
from pathlib import Path


def find_project_root() -> Path:
    """
    pyproject.toml 파일을 기준으로 루트 디렉토리를 찾는다.
    :return: Path: 프로젝트 루트 디렉토리 경로
    """

    current_path = Path().resolve()

    while current_path != current_path.parent:
        if (current_path / ROOT_MARKER).exists():
            return current_path

        current_path = current_path.parent

    raise FileNotFoundError("프로젝트 루트 디렉토리를 찾을 수 없습니다.")


ROOT_DIR = find_project_root()

In [3]:
# matplotlib 의 한글 font 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt


FONTS_DATA_DIR = ROOT_DIR / "notebooks" / "fonts"


def setup_korean_font():
    font_path = FONTS_DATA_DIR / KOREAN_FONT_FILE
    fm.fontManager.addfont(font_path)

    # 폰트 설정
    plt.rcParams["font.family"] = KOREAN_FONT_FAMILY
    plt.rcParams["axes.unicode_minus"] = False


setup_korean_font()

# 기상 데이터 수집 함수 작성

In [4]:
from datetime import datetime, timedelta

from dateutil.relativedelta import relativedelta

from src.utils.config import timezone


today = datetime.now(timezone)
end_date = today - timedelta(days=1)
start_date = end_date - relativedelta(years=1)

In [5]:
print("start_date:", start_date)
print("end_date:", end_date)

start_date: 2024-05-28 16:42:17.443962+09:00
end_date: 2025-05-28 16:42:17.443962+09:00


In [6]:
import pandas as pd


asos_df_list: list[pd.DataFrame] = []

In [7]:
from dataclasses import asdict

from src.libs.weather.asosstation import AsosStation
from src.libs.weather.fetcher import AsosDataFetcher


fetcher = AsosDataFetcher.create()

asos_data: dict[AsosStation : pd.DataFrame] = {}

for station in AsosStation:
    res = fetcher.fetch_all(
        asos_station=station,
        start_date=start_date,
        end_date=end_date,
    )
    print(f"total asos data of {station.name}:", len(res.items))

    asos_data[station] = pd.DataFrame([asdict(item) for item in res.items])
    break

[2025-05-29 16:42:19] INFO [src.libs.weather.vo.create] Success to load ASOS data.


total asos data of SOKCHO: 366


In [8]:
from src.libs.storage import Storage


storage = Storage.create()

In [9]:
from src.data.collector import Collector


collector = Collector(storage, fetcher)
collector.collect_all_asos_data(start_date, end_date)
# collector.upload_all_asos_data()

[2025-05-29 16:42:20] INFO [src.libs.weather.vo.create] Success to load ASOS data.
[2025-05-29 16:42:22] INFO [src.libs.weather.vo.create] Success to load ASOS data.
[2025-05-29 16:42:23] INFO [src.libs.weather.vo.create] Success to load ASOS data.
[2025-05-29 16:42:24] INFO [src.libs.weather.vo.create] Success to load ASOS data.
[2025-05-29 16:42:25] INFO [src.libs.weather.vo.create] Success to load ASOS data.
[2025-05-29 16:42:26] INFO [src.libs.weather.vo.create] Success to load ASOS data.
[2025-05-29 16:42:27] INFO [src.libs.weather.vo.create] Success to load ASOS data.
[2025-05-29 16:42:29] INFO [src.libs.weather.vo.create] Success to load ASOS data.
[2025-05-29 16:42:30] INFO [src.libs.weather.vo.create] Success to load ASOS data.
[2025-05-29 16:42:31] INFO [src.libs.weather.vo.create] Success to load ASOS data.
[2025-05-29 16:42:32] INFO [src.libs.weather.vo.create] Success to load ASOS data.
[2025-05-29 16:42:33] INFO [src.libs.weather.vo.create] Success to load ASOS data.
[202

['datasets/1748504656530-20240528-20250528-90-sokcho.csv',
 'datasets/1748504656665-20240528-20250528-93-north_chuncheon.csv',
 'datasets/1748504656790-20240528-20250528-95-cheorwon.csv',
 'datasets/1748504656883-20240528-20250528-98-dongducheon.csv',
 'datasets/1748504656998-20240528-20250528-99-paju.csv',
 'datasets/1748504657103-20240528-20250528-100-daegwallyeong.csv',
 'datasets/1748504657204-20240528-20250528-101-chuncheon.csv',
 'datasets/1748504657316-20240528-20250528-102-baengnyeongdo.csv',
 'datasets/1748504657442-20240528-20250528-104-north_gangneung.csv',
 'datasets/1748504657583-20240528-20250528-105-gangneung.csv',
 'datasets/1748504657680-20240528-20250528-106-donghae.csv',
 'datasets/1748504657768-20240528-20250528-108-seoul.csv',
 'datasets/1748504657867-20240528-20250528-112-incheon.csv',
 'datasets/1748504657999-20240528-20250528-114-wonju.csv',
 'datasets/1748504658104-20240528-20250528-115-ulleungdo.csv',
 'datasets/1748504658246-20240528-20250528-119-suwon.csv',


In [10]:
from src.data.loader import AsosDataLoader


loader = AsosDataLoader(storage)
asos_station_dfs = loader.load()

[2025-05-29 16:44:25] INFO [src.libs.storage._check_and_log_response] Success to retrieve 
[2025-05-29 16:44:25] INFO [src.libs.storage._check_and_log_response] Success to read datasets/20200527-20250527-100-daegwallyeong.csv
[2025-05-29 16:44:25] INFO [src.libs.storage._check_and_log_response] Success to read datasets/20200527-20250527-101-chuncheon.csv
[2025-05-29 16:44:25] INFO [src.libs.storage._check_and_log_response] Success to read datasets/20200527-20250527-102-baengnyeongdo.csv
[2025-05-29 16:44:25] INFO [src.libs.storage._check_and_log_response] Success to read datasets/20200527-20250527-105-gangneung.csv
[2025-05-29 16:44:25] INFO [src.libs.storage._check_and_log_response] Success to read datasets/20200527-20250527-106-donghae.csv
[2025-05-29 16:44:26] INFO [src.libs.storage._check_and_log_response] Success to read datasets/20200527-20250527-108-seoul.csv
[2025-05-29 16:44:26] INFO [src.libs.storage._check_and_log_response] Success to read datasets/20200527-20250527-112-inch

In [11]:
for station, df in asos_station_dfs.items():
    print(station, df.shape)

AsosStation.DAEGWALLYEONG (1827, 62)
AsosStation.CHUNCHEON (1826, 62)
AsosStation.BAENGNYEONGDO (1827, 62)
AsosStation.GANGNEUNG (1827, 62)
AsosStation.DONGHAE (1827, 62)
AsosStation.SEOUL (1827, 62)
AsosStation.INCHEON (1827, 62)
AsosStation.WONJU (1827, 62)
AsosStation.ULLEUNGDO (1827, 62)
AsosStation.SUWON (1827, 62)
AsosStation.YEONGWOL (1827, 62)
AsosStation.CHUNGJU (1827, 62)
AsosStation.SEOSAN (1827, 62)
AsosStation.ULJIN (1827, 62)
AsosStation.CHEONGJU (1827, 62)
AsosStation.DAEJEON (1827, 62)
AsosStation.CHUPUNGNYEONG (1827, 62)
AsosStation.ANDONG (1827, 62)
AsosStation.SANGJU (1827, 62)
AsosStation.POHANG (1827, 62)
AsosStation.GUNSAN (1826, 62)
AsosStation.DAEGU (1827, 62)
AsosStation.JEONJU (1827, 62)
AsosStation.ULSAN (1827, 62)
AsosStation.CHANGWON (1827, 62)
AsosStation.GWANGJU (1827, 62)
AsosStation.BUSAN (1827, 62)
AsosStation.TONGYEONG (1825, 62)
AsosStation.MOKPO (1827, 62)
AsosStation.YEOSU (1827, 62)
AsosStation.HEUKSANDO (1827, 62)
AsosStation.WANDO (1827, 62)
Aso

In [12]:
import pandas as pd


pd.concat(asos_station_dfs.values())

,stn_id,stn_nm,tm,avg_ta,min_ta,min_ta_hrmt,max_ta,max_ta_hrmt,sum_rn_dur,mi10_max_rn,...,avg_m05_te,avg_m10_te,avg_m15_te,avg_m30_te,avg_m50_te,sum_lrg_ev,sum_sml_ev,n99_rn,iscs,sum_fog_dur
0,100,대관령,2020-05-27,13.4,8.0,529.0,19.2,1332.0,NaN,0.1,...,NaN,NaN,NaN,NaN,NaN,3.8,5.4,NaN,NaN,NaN
1,100,대관령,2020-05-28,12.3,7.0,243.0,19.4,1144.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,3.4,4.8,NaN,NaN,NaN
2,100,대관령,2020-05-29,12.6,4.4,550.0,19.9,1207.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.7,6.7,NaN,NaN,NaN
3,100,대관령,2020-05-30,14.1,4.0,503.0,22.8,1426.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.7,8.2,NaN,NaN,NaN
4,100,대관령,2020-05-31,16.5,5.6,453.0,22.3,1058.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.4,6.3,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,99,파주,2025-05-23,16.5,13.6,2346.0,21.9,1425.0,NaN,0.5,...,NaN,NaN,NaN,NaN,NaN,1.7,2.4,NaN,NaN,NaN
1823,99,파주,2025-05-24,14.8,9.6,2333.0,19.5,1248.0,NaN,0.1,...,NaN,NaN,NaN,NaN,NaN,2.7,3.8,NaN,NaN,NaN
1824,99,파주,2025-05-25,15.5,7.6,520.0,22.3,1533.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.2,6.1,NaN,NaN,NaN
1825,99,파주,2025-05-26,16.7,9.0,526.0,25.3,1501.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.2,6.0,NaN,NaN,NaN
